<a href="https://colab.research.google.com/github/VNagaHarshitha/SentimentAnalysisProject/blob/main/sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install slack_sdk

import requests
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

#  API Keys (stored in Colab Secrets)
NEWSAPI_KEY = userdata.get('NEWSAPI_KEY')
GNEWS_KEY = userdata.get('GNEWS_KEY')
GEMINI_KEY = userdata.get('GEMINI_API_KEY')
SLACK_WEBHOOK_URL = userdata.get('SLACK_WEBHOOK_URL')  # Webhook URL for Slack

#  Configure Gemini
try:
    genai.configure(api_key=GEMINI_KEY)
    model = genai.GenerativeModel("gemini-1.5-flash")
except Exception as e:
    print(" Could not initialize Gemini:", e)
    model = None

#  Fetch News
def fetch_newsapi(query="business", max_articles=50):
    url = "https://newsapi.org/v2/everything"
    params = {"q": query, "language": "en", "pageSize": max_articles, "apiKey": NEWSAPI_KEY}
    r = requests.get(url, params=params).json()
    return [{"source": "NewsAPI", "title": a.get("title", ""), "description": a.get("description", "")} for a in r.get("articles", [])]


def fetch_gnews(query="business", max_articles=50):
    url = "https://gnews.io/api/v4/search"
    params = {"q": query, "lang": "en", "max": max_articles, "token": GNEWS_KEY}
    r = requests.get(url, params=params).json()
    return [{"source": "GNews", "title": a.get("title", ""), "description": a.get("description", "")} for a in r.get("articles", [])]

#  Sentiment Analysis
def analyze_sentiments(texts, batch_size=100):
    sentiments = []
    if not model:
        return ["Neutral"] * len(texts)
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        joined = "\n".join([f"[{j+1}] {t}" for j, t in enumerate(batch)])
        prompt = f"""
        Classify each news headline as Positive, Negative, or Neutral.
        Respond only in this format:
        [1] Positive
        [2] Neutral
        [3] Negative

        Headlines:
        {joined}
        """
        try:
            response = model.generate_content(prompt, request_options={"timeout": 20})
            raw = response.text.strip()
            lines = [line.split("]")[-1].strip() for line in raw.split("\n") if line.strip()]
            batch_sentiments = [s if s in ["Positive", "Negative", "Neutral"] else "Neutral" for s in lines]
            sentiments.extend(batch_sentiments)
        except Exception:
            sentiments.extend(["Neutral"] * len(batch))
    return sentiments


#  Slack Alerts
def send_slack_alert(message: str):
    url = SLACK_WEBHOOK_URL
    if not url:
        print(" Slack Webhook URL not set in Colab secrets.")
        return
    payload = {"text": message}
    try:
        r = requests.post(url, json=payload)
        if r.status_code == 200:
            print(" Slack alert sent")
        else:
            print(f"Failed to send Slack alert: {r.status_code}, {r.text}")
    except Exception as e:
        print(" Slack alert error:", e)



#  Main

if __name__ == "__main__":
    articles = fetch_newsapi("stock market", 50) + fetch_gnews("stock market", 50)

    if articles:
        texts = [a["title"] for a in articles if a["title"]]
        sentiments = analyze_sentiments(texts)

        for i in range(len(articles)):
            articles[i]["sentiment"] = sentiments[i] if i < len(sentiments) else "Neutral"

        df = pd.DataFrame(articles)
        df.to_csv("combined_news_sentiment.csv", index=False)

        #  Filter positive news only
        positive_news = df[df["sentiment"] == "Positive"]

        if not positive_news.empty:
            for _, row in positive_news.iterrows():
                msg = f" *Positive News*\n*Source:* {row['source']}\n*Title:* {row['title']}\n*Description:* {row['description']}"
                send_slack_alert(msg)
            print(f"Sent {len(positive_news)} positive news alerts to Slack")
        else:
            send_slack_alert("ℹ No positive news found today.")

    else:
        print(" No articles found.")
        send_slack_alert(" No articles found for query.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.9/293.9 kB 5.5 MB/s eta 0:00:00
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
 Slack alert sent
Sent 15 positive news alerts to Slack
